In [2]:
import requests
import json
import pandas as pd
import time
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
client_id = os.environ.get('CLIENT_ID')
client_secret = os.environ.get('CLIENT_SECRET')

In [5]:
def get_oauth_token(client_id = client_id, client_secret = client_secret):
    url = 'https://id.twitch.tv/oauth2/token'
    query_string = {'client_id': client_id, 
                    'client_secret': client_secret,
                    'grant_type': 'client_credentials'}
    r = requests.post(url, params=query_string).json()
    return r['access_token']

In [6]:
def get_top_100_categories(client_id = client_id):
    bearer_token = get_oauth_token()
    headers = {'Client-ID': client_id, 
               'Authorization': 'Bearer ' + bearer_token}
    url = r'https://api.twitch.tv/helix/games/top?first=100'
    r = requests.get(url, headers=headers)
    return r

In [7]:
def check_rate_limit_reached(req, ignore_limit = False):
    # checks if there is 1 request before rate limiting
    if int(req.headers['Ratelimit-Remaining']) <= 1:
        # not really preferred could cause trouble
        if ignore_limit:
            return int(req.headers['Ratelimit-Remaining'])
        print('Rate limit refreshes in 30s')
        time.sleep(30)
        print('Ready!')
    # return the remaining tries
    return int(req.headers['Ratelimit-Remaining'])

In [8]:
def get_top_100_streamers_for_each_game(games):
    streamers = {}
    
    bearer_token = get_oauth_token()
    headers = {'Client-ID': client_id, 
               'Authorization': 'Bearer ' + bearer_token}
    url = 'https://api.twitch.tv/helix/streams?first=100&game_id='
    
    for game in games['data']:
        req = requests.get(url + game['id'], headers=headers)
        check_rate_limit_reached(req)
        streamers[game['name']] = json.loads(req.text)
    return streamers

In [9]:
def json_to_df(json):
    total_streams_df = pd.DataFrame(
        columns = ['game_id','id','language','started_at','title','type','user_id','user_name','viewer_count'])
    for game_key in list(json.keys()):
        game_streams_df = pd.json_normalize(json[game_key]['data'])
        total_streams_df = pd.concat([total_streams_df, game_streams_df], sort = False)
    total_streams_df.drop(columns = ['thumbnail_url','tag_ids'], inplace = True)
    return total_streams_df

In [13]:
top_100_games = get_top_100_categories().json()
top_streamers = get_top_100_streamers_for_each_game(top_100_games)

In [14]:
df = json_to_df(top_streamers)
df.head(10)

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name
0,509658,42298686670,en,2021-04-12T19:49:05Z,THE LAST MODCAST 4REALZ THIS TIME :( | !SUBAT...,live,40934651,ludwig,43599,ludwig,Just Chatting
1,509658,42305684622,es,2021-04-13T12:57:56Z,IRL TIER LIST MEJORES YOGURES DE ESPAÑA,live,199046842,nilojeda,6954,nilojeda,Just Chatting
2,509658,41753830156,pl,2021-04-13T13:08:16Z,SZYBKO! SZYBKO! SZYBKO! | !dc !txt,live,69616738,xth0rek,6166,xth0rek,Just Chatting
3,509658,41753763388,it,2021-04-13T13:00:30Z,MARRA SBBUTTANATO DA UNA RAGAZZA,live,264708119,Cerbero_Podcast,5754,cerbero_podcast,Just Chatting
4,509658,41413182205,es,2021-04-13T13:22:25Z,HOY STREAM TODO EL DIA | 545/700 subs,live,143737983,Pimpeano,5200,pimpeano,Just Chatting
5,509658,42305247150,en,2021-04-13T11:00:31Z,Adventure Day 0 !,live,159498717,Jinnytty,5112,jinnytty,Just Chatting
6,509658,41413205181,en,2021-04-13T13:25:25Z,"I'm a new CEO, AMA! | !invest !platform !coffe...",live,249820319,TheStockGuy,4525,thestockguy,Just Chatting
7,509658,42304945198,ko,2021-04-13T09:23:31Z,설문놀이 : 자동차편,live,49045679,우왁굳,4351,woowakgood,Just Chatting
8,509658,41411634797,ko,2021-04-13T09:03:18Z,몰래몰래온 송파구의 자랑,live,42848761,자동,4224,tranth,Just Chatting
9,509658,41411381837,ru,2021-04-13T08:00:55Z,МАРАФОН ОРЛА И РЕШКИ 10 ЧАСОВ И БОЛЬШЕ НИКОГДА...,live,188890121,Dmitry_Lixxx,4132,dmitry_lixxx,Just Chatting


In [15]:
df.tail(10)

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name
90,512980,41412383373,en,2021-04-13T11:30:01Z,GUYS ARE FALLING. LETS WIN AND LOSE.,live,157766584,SmexyLump,6,smexylump,Fall Guys: Ultimate Knockout
91,512980,41413135597,pl,2021-04-13T13:16:15Z,"1192 wins fasoleczki, !Donate !discord !gramy ...",live,522140215,Beteku,6,beteku,Fall Guys: Ultimate Knockout
92,512980,42305095038,de,2021-04-13T10:11:27Z,Alo Alo wasn los hier,live,534052651,Brate_Jo,6,brate_jo,Fall Guys: Ultimate Knockout
93,512980,42305567134,fr,2021-04-13T12:28:18Z,[PS4/FR]duo avec ma tite cece <3 - !discord ...,live,229050703,azzou_,6,azzou_,Fall Guys: Ultimate Knockout
94,512980,41753485468,de,2021-04-13T12:21:11Z,ausschließlich gute plays,live,213003932,ALNEXY,6,alnexy,Fall Guys: Ultimate Knockout
95,512980,42305645582,ja,2021-04-13T12:48:22Z,FALLGYUS！1位取るまで終わらないこともない(';')1jikanndake...,live,463625480,inachan,5,inachan,Fall Guys: Ultimate Knockout
96,512980,41754085020,fr,2021-04-13T13:41:10Z,Reprise des tournois en partie privée Samedi !...,live,262341150,LePirateRoux,5,lepirateroux,Fall Guys: Ultimate Knockout
97,512980,41412661549,ja,2021-04-13T12:11:22Z,【視聴者参加型】登りすぎたので落ちる．参加してくれる人はチャットで教えてね！！（PC）,live,170671669,watt_W,5,watt_w,Fall Guys: Ultimate Knockout
98,512980,41412647709,en,2021-04-13T12:09:21Z,Putting the Spicy Back In the Beans. !merch ...,live,51669105,Jalapep,5,jalapep,Fall Guys: Ultimate Knockout
99,512980,41412539165,pt,2021-04-13T11:54:50Z,O CARA CAIU DA ESCADA- Primepls(coroinha samba...,live,184730056,LUQUINHA5,5,luquinha5,Fall Guys: Ultimate Knockout


In [16]:
df.to_csv('./data/top_live_streamers.csv')